# AiiDA WorkChain Example for Material Research Process (Refined)

This notebook demonstrates how to implement and use AiiDA WorkChains to represent a parallel binder characterization, independent calibration/validation of DM and MS models, and an upscaling process, as described in the refined state diagram.

We will:
- Define calculation functions and WorkChains for each major step, including upscaling.
- Show how to submit, monitor, and retrieve results.
- Annotate each step for clarity and learning.

**Note:** This notebook assumes you have AiiDA installed and a profile configured. For a real lab, plugins for experimental data import, simulation, and upscaling would need to be developed.

In [ ]:
# Import AiiDA core modules and set up the environment
from aiida.engine import WorkChain, calcfunction, run, submit
from aiida.orm import Dict, Str
from aiida.manage import get_manager

# Ensure AiiDA is loaded and a profile is active
get_manager().get_profile()

## 1. Define Calculation Functions

These are simple stand-ins for real calculations (e.g., data import, model calibration, simulation, upscaling).
In a real project, these would be replaced by plugins that run actual codes or data processing scripts.

In [ ]:
@calcfunction
def import_experimental_data(binder: Str, test_type: Str) -> Dict:
    """Mock function to import experimental data for a binder and test type."""
    # In practice, this would load data from files or a database
    return Dict(dict={
        'binder': binder.value,
        'test_type': test_type.value,
        'data': f"mock_data_{binder.value}_{test_type.value}"
    })

@calcfunction
def calibrate_model(model_type: Str, exp_data: Dict) -> Dict:
    """Mock function to calibrate a model using experimental data."""
    return Dict(dict={
        'model_type': model_type.value,
        'calibrated_params': f"params_for_{model_type.value}_{exp_data['test_type']}"
    })

@calcfunction
def run_validation(model_type: Str, params: Dict, validation_data: Dict) -> Dict:
    """Mock function to validate a model."""
    return Dict(dict={
        'model_type': model_type.value,
        'validation_result': f"validated_{model_type.value}_on_{validation_data['test_type']}"
    })

@calcfunction
def upscaling_dm_to_ms(dm_params: Dict) -> Dict:
    """Mock function to upscale DM parameters to MS parameters using energetic equivalence."""
    return Dict(dict={
        'upscaled_ms_params': f"upscaled_from_{dm_params['calibrated_params']}"
    })

## 2. Define WorkChains for Each Major State

Each WorkChain represents a phase in the research process. DM and MS calibration/validation are independent, and upscaling is a separate process.

In [ ]:
from aiida.engine import ToContext

class DMCalibrationWorkChain(WorkChain):
    """WorkChain for calibrating and validating the Discrete Model (DM)."""
    @classmethod
    def define(cls, spec):
        super().define(spec)
        spec.input('binder', valid_type=Str)
        spec.input('test_type', valid_type=Str)
        spec.input('validation_type', valid_type=Str)
        spec.outline(
            cls.import_data,
            cls.calibrate,
            cls.validate,
        )
        spec.output('dm_params', valid_type=Dict)
        spec.output('dm_validation', valid_type=Dict)

    def import_data(self):
        self.ctx.exp_data = import_experimental_data(self.inputs.binder, self.inputs.test_type)

    def calibrate(self):
        self.ctx.dm_params = calibrate_model(Str('DM'), self.ctx.exp_data)

    def validate(self):
        validation_data = import_experimental_data(self.inputs.binder, self.inputs.validation_type)
        dm_val = run_validation(Str('DM'), self.ctx.dm_params, validation_data)
        self.out('dm_params', self.ctx.dm_params)
        self.out('dm_validation', dm_val)

class MSCalibrationWorkChain(WorkChain):
    """WorkChain for calibrating and validating the Microplane Model (MS)."""
    @classmethod
    def define(cls, spec):
        super().define(spec)
        spec.input('binder', valid_type=Str)
        spec.input('test_type', valid_type=Str)
        spec.input('validation_type', valid_type=Str)
        spec.outline(
            cls.import_data,
            cls.calibrate,
            cls.validate,
        )
        spec.output('ms_params', valid_type=Dict)
        spec.output('ms_validation', valid_type=Dict)

    def import_data(self):
        self.ctx.exp_data = import_experimental_data(self.inputs.binder, self.inputs.test_type)

    def calibrate(self):
        self.ctx.ms_params = calibrate_model(Str('MS'), self.ctx.exp_data)

    def validate(self):
        validation_data = import_experimental_data(self.inputs.binder, self.inputs.validation_type)
        ms_val = run_validation(Str('MS'), self.ctx.ms_params, validation_data)
        self.out('ms_params', self.ctx.ms_params)
        self.out('ms_validation', ms_val)

class UpscalingWorkChain(WorkChain):
    """WorkChain for upscaling DM parameters to MS parameters."""
    @classmethod
    def define(cls, spec):
        super().define(spec)
        spec.input('dm_params', valid_type=Dict)
        spec.outline(
            cls.upscale,
        )
        spec.output('upscaled_ms_params', valid_type=Dict)

    def upscale(self):
        upscaled = upscaling_dm_to_ms(self.inputs.dm_params)
        self.out('upscaled_ms_params', upscaled)

## 3. Compose a High-Level WorkChain

This WorkChain orchestrates the full process: independent calibration/validation of DM and MS, and upscaling from DM to MS.

In [ ]:
class MaterialResearchWorkChain(WorkChain):
    """High-level WorkChain for the full material research process."""
    @classmethod
    def define(cls, spec):
        super().define(spec)
        spec.input('binder', valid_type=Str)
        spec.input('calib_test_type', valid_type=Str)
        spec.input('validation_type', valid_type=Str)
        spec.outline(
            cls.calibrate_dm,
            cls.calibrate_ms,
            cls.upscale_dm_to_ms,
            cls.finalize,
        )
        spec.output('dm_params', valid_type=Dict)
        spec.output('dm_validation', valid_type=Dict)
        spec.output('ms_params', valid_type=Dict)
        spec.output('ms_validation', valid_type=Dict)
        spec.output('upscaled_ms_params', valid_type=Dict)

    def calibrate_dm(self):
        dm = self.submit(DMCalibrationWorkChain,
                         binder=self.inputs.binder,
                         test_type=self.inputs.calib_test_type,
                         validation_type=self.inputs.validation_type)
        return ToContext(dm=dm)

    def calibrate_ms(self):
        ms = self.submit(MSCalibrationWorkChain,
                         binder=self.inputs.binder,
                         test_type=self.inputs.calib_test_type,
                         validation_type=self.inputs.validation_type)
        return ToContext(ms=ms)

    def upscale_dm_to_ms(self):
        upscaled = self.submit(UpscalingWorkChain,
                               dm_params=self.ctx.dm.outputs.dm_params)
        return ToContext(upscaled=upscaled)

    def finalize(self):
        self.out('dm_params', self.ctx.dm.outputs.dm_params)
        self.out('dm_validation', self.ctx.dm.outputs.dm_validation)
        self.out('ms_params', self.ctx.ms.outputs.ms_params)
        self.out('ms_validation', self.ctx.ms.outputs.ms_validation)
        self.out('upscaled_ms_params', self.ctx.upscaled.outputs.upscaled_ms_params)

## 4. Submit and Monitor the Workflow

Now, let's submit the high-level workflow and monitor its progress.

In [ ]:
# Define input parameters for the workflow
binder = Str('BinderX')
calib_test_type = Str('CT_Monotonic')
validation_type = Str('CT_Fatigue')

# Submit the workflow (asynchronously)
from aiida.engine import submit
future = submit(MaterialResearchWorkChain,
                binder=binder,
                calib_test_type=calib_test_type,
                validation_type=validation_type)

print(f"Submitted workflow with PK={future.pk}")

### Monitoring and Retrieving Results

You can monitor the workflow in the AiiDA shell or using verdi commands:

```bash
verdi process list -a
verdi process show <PK>
```

Or, in Python:

In [ ]:
from aiida.orm import load_node
import time

# Wait for the workflow to finish (for demonstration; in production, use event-driven or verdi)
while not future.is_terminated:
    print(f"Workflow state: {future.process_state}")
    time.sleep(2)

# Retrieve outputs
node = load_node(future.pk)
print("DM Calibration Params:", node.outputs.dm_params.get_dict())
print("DM Validation Result:", node.outputs.dm_validation.get_dict())
print("MS Calibration Params:", node.outputs.ms_params.get_dict())
print("MS Validation Result:", node.outputs.ms_validation.get_dict())
print("Upscaled MS Params:", node.outputs.upscaled_ms_params.get_dict())

## 5. Summary

This notebook illustrated how to map a refined material research process (with independent model calibration/validation and upscaling) into AiiDA WorkChains. In a real project, you would replace the mock calculation functions with plugins that run actual experiments, simulations, or data processing scripts.

AiiDA's provenance, parallelism, and workflow management make it a strong fit for such research automation and documentation tasks.